In [ ]:
!pip install -q transformers

     |████████████████████████████████| 2.6 MB 14.8 MB/s 
     |████████████████████████████████| 636 kB 59.4 MB/s 
     |████████████████████████████████| 895 kB 74.5 MB/s 
     |████████████████████████████████| 3.3 MB 55.8 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%matplotlib inline
import os
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import classification_report
import re
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
import transformers
from transformers import RobertaTokenizer, BertTokenizer, RobertaModel, BertModel, AdamW# get_linear_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup
import time

!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/utils.py .
from utils import *
!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/Custom_Dataset_Class.py .
from Custom_Dataset_Class import CustomDataset
!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/pytorchtools.py .
from pytorchtools import EarlyStopping
#from Bert_Classification import Bert_Classification_Model
#from RoBERT import RoBERT_Model

#from BERT_Hierarchical import BERT_Hierarchical_Model
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelBinarizer

In [ ]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed_all(123)

In [ ]:
#change to where you store mimic3 data
MIMIC_3_DIR = '/content/drive/MyDrive/Colab Notebooks/MSc-Individual-Project/datasets'

train_df = pd.read_csv('%s/train_10.csv' % MIMIC_3_DIR)
eval_df = pd.read_csv('%s/dev_10.csv' % MIMIC_3_DIR)
test_df = pd.read_csv('%s/test_10.csv' % MIMIC_3_DIR)

train_df.head()

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,LABELS,length
0,17341,151110,Nursing/other,rsbi,584.9;427.31,1
1,61638,103816,Nursing,title,414.01,1
2,61638,103816,General,title,414.01,1
3,23706,186321,Nursing/other,npn,401.9;428.0;530.81,1
4,55265,191108,General,title,530.81;584.9;427.31,1


In [ ]:
full_df = pd.concat([train_df, eval_df, test_df], ignore_index=True)

In [ ]:
# split labels by ";", then convert to list
def split_lab (x):
    #print(x)
    return x.split(";")

full_df['LABELS'] = full_df['LABELS'].apply(split_lab)
#full_df['TEXT'] = full_df['TEXT'].apply(split_lab)

full_df.head()

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,LABELS,length
0,17341,151110,Nursing/other,rsbi,"[584.9, 427.31]",1
1,61638,103816,Nursing,title,[414.01],1
2,61638,103816,General,title,[414.01],1
3,23706,186321,Nursing/other,npn,"[401.9, 428.0, 530.81]",1
4,55265,191108,General,title,"[530.81, 584.9, 427.31]",1


In [ ]:
#load multi label binarizer for one-hot encoding
mlb = MultiLabelBinarizer(sparse_output=True)

#labels_onehot = mlb.fit_transform(train_df.pop('LABELS'))
#labels_onehot[0][1]

In [ ]:
#change label to one-hot encoding per code
full_df = full_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(full_df.pop('LABELS')),
                columns=mlb.classes_))

full_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,250.00,272.4,401.9,414.01,427.31,428.0,518.81,530.81,584.9,599.0
0,17341,151110,Nursing/other,rsbi,1,0,0,0,0,1,0,0,0,1,0
1,61638,103816,Nursing,title,1,0,0,0,1,0,0,0,0,0,0
2,61638,103816,General,title,1,0,0,0,1,0,0,0,0,0,0
3,23706,186321,Nursing/other,npn,1,0,0,1,0,0,1,0,1,0,0
4,55265,191108,General,title,1,0,0,0,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294460,97158,152158,Discharge summary,admission date discharge date date of birth se...,4644,0,0,1,0,0,0,1,0,0,0
294461,99650,199859,Discharge summary,admission date discharge date date of birth se...,5126,0,0,0,1,1,1,1,0,1,1
294462,93623,187232,Discharge summary,admission date discharge date date of birth se...,5171,0,1,1,0,0,0,0,0,0,0
294463,96260,110058,Discharge summary,admission date discharge date date of birth se...,5173,0,0,0,0,0,0,1,0,0,1


In [ ]:
# Convert columns to list of one hot encoding
icd_classes_50 = mlb.classes_

full_df['labels'] = full_df[icd_classes_50].values.tolist()
#train_df.sort_values(['length'], ascending=False, inplace=True)
full_df.head()


,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,250.00,272.4,401.9,414.01,427.31,428.0,518.81,530.81,584.9,599.0,labels
0,17341,151110,Nursing/other,rsbi,1,0,0,0,0,1,0,0,0,1,0,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0]"
1,61638,103816,Nursing,title,1,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
2,61638,103816,General,title,1,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
3,23706,186321,Nursing/other,npn,1,0,0,1,0,0,1,0,1,0,0,"[0, 0, 1, 0, 0, 1, 0, 1, 0, 0]"
4,55265,191108,General,title,1,0,0,0,0,1,0,0,1,1,0,"[0, 0, 0, 0, 1, 0, 0, 1, 1, 0]"


In [ ]:
full_df = full_df.drop(full_df[full_df['length']<300].index)

In [ ]:
full_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,250.00,272.4,401.9,414.01,427.31,428.0,518.81,530.81,584.9,599.0,labels
155040,65582,194117,Nursing,name8 md md h p yo male with strep bovis endoc...,300,0,0,0,0,0,0,1,0,0,1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
155041,70924,119848,Nursing,42f with chronic alcoholic pancreatitis transf...,300,0,0,0,0,0,0,1,0,1,1,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1]"
155042,25139,180782,Radiology,am picc line placment sch clip clip number rad...,300,0,0,0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
155043,13494,164195,Radiology,pm ct abdomen w o contrast ct pelvis w o contr...,300,0,0,0,0,0,0,1,0,0,1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
155044,30275,152118,Radiology,pm perc g g j tube plmt clip clip number radio...,300,0,0,0,0,0,0,1,0,0,1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294460,97158,152158,Discharge summary,admission date discharge date date of birth se...,4644,0,0,1,0,0,0,1,0,0,0,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]"
294461,99650,199859,Discharge summary,admission date discharge date date of birth se...,5126,0,0,0,1,1,1,1,0,1,1,"[0, 0, 0, 1, 1, 1, 1, 0, 1, 1]"
294462,93623,187232,Discharge summary,admission date discharge date date of birth se...,5171,0,1,1,0,0,0,0,0,0,0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]"
294463,96260,110058,Discharge summary,admission date discharge date date of birth se...,5173,0,0,0,0,0,0,1,0,0,1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"


In [ ]:
train_df, test_df = train_test_split(full_df, test_size=0.2)

In [ ]:
train_df, dev_df = train_test_split(train_df, test_size=0.2)

In [ ]:
train_df.sort_values(['length'], inplace=True)
dev_df.sort_values(['length'], inplace=True)
test_df.sort_values(['length'], inplace=True)


In [ ]:
#convert into 2 columns dataframe
train_df = pd.DataFrame(train_df, columns=['TEXT', 'labels'])
train_df.columns=['text', 'labels']
train_df.head()

dev_df = pd.DataFrame(dev_df, columns=['HADM_ID', 'TEXT', 'labels'])
dev_df.columns=['id', 'text', 'labels']
dev_df.head()

test_df = pd.DataFrame(test_df, columns=['HADM_ID', 'TEXT', 'labels'])
test_df.columns=['id', 'text', 'labels']
test_df.head()

,id,text,labels
276299,111336,year old male admitted s p fall from a ladder ...,"[0, 1, 1, 0, 0, 0, 1, 0, 0, 0]"
155161,192003,this is a year old female with a history of ci...,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0]"
155053,153856,am last name un intestinal tube placement w fl...,"[0, 1, 1, 0, 1, 1, 0, 0, 0, 0]"
155149,181811,coronary artery bypass graft cabg assessment p...,"[0, 1, 1, 1, 1, 1, 0, 0, 0, 0]"
155075,117776,nursing progress note pt continues to have res...,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0]"


In [ ]:
train_df.reset_index(drop=True, inplace=True)
dev_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
test_df.head()

,id,text,labels
0,111336,year old male admitted s p fall from a ladder ...,"[0, 1, 1, 0, 0, 0, 1, 0, 0, 0]"
1,192003,this is a year old female with a history of ci...,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0]"
2,153856,am last name un intestinal tube placement w fl...,"[0, 1, 1, 0, 1, 1, 0, 0, 0, 0]"
3,181811,coronary artery bypass graft cabg assessment p...,"[0, 1, 1, 1, 1, 1, 0, 0, 0, 0]"
4,117776,nursing progress note pt continues to have res...,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0]"


In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        '''
            Load Pretrained model here
            Use return_dict=False for compatibility for 4.x

        '''
        self.l1 = transformers.AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", return_dict=False)
        #self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)


        self.l2 = torch.nn.Dropout(0.3)

        self.l3 = torch.nn.Linear(768, 10)

    def forward(self, ids, mask, token_type_ids):
#        print("ids: ", ids.size(), "mask: ", mask.size(), "token type ids: ", token_type_ids.size())
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [ ]:
# Defining some key variables to configure model training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 12
VALID_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 3e-05

#set tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

#custom dataset for BERT class
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        
        '''
            set text as training data
            set labels as targets
        '''
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }



Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [ ]:
#load df to dataset

training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
dev_set = CustomDataset(dev_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

In [ ]:
#data loader
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': False
                }


dev_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': False
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': False
                }

training_loader = DataLoader(training_set, **train_params)
dev_loader = DataLoader(dev_set, **dev_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
#loss function
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

#optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        
    print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

In [ ]:
# Evaluate the model

def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    losses=[]
    with torch.no_grad():
        for _, data in enumerate(dev_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    print(f'Epoch: {epoch}, Validation Loss:  {np.mean(losses):.2f}')
    return fin_outputs, fin_targets, losses

In [ ]:
start_epoch=0
DIR = '/content/drive/MyDrive/Colab Notebooks/MSc-Individual-Project/'
resume = True     
if resume:
    if os.path.isfile(f"%s/models/model_allnotesshuffle10_epoch{start_epoch}.pth" % DIR):
        print("Resume from checkpoint...")
        checkpoint = torch.load(f"%s/models/model_allnotesshuffle10_epoch{start_epoch}.pth" % DIR)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        initepoch = checkpoint['epoch']
        print("====>loaded checkpoint (epoch{})".format(checkpoint['epoch']))
    else:
        print("====>no checkpoint found.")
        initepoch = 0

#patience = 3
#early_stopping = EarlyStopping(patience, verbose=True)


for epoch in tqdm(range(EPOCHS)):
    train(epoch)
    validation(epoch)

    if (epoch+start_epoch+1)%5 == 0:
        checkpoint = {"model_state_dict": model.state_dict(),
                      "optimizer_state_dict": optimizer.state_dict(),
                      "epoch": epoch+start_epoch+1}
        path_checkpoint = f"%s/models/model_allnotesshuffle10_epoch{epoch+start_epoch+1}.pth" % DIR
        torch.save(checkpoint, path_checkpoint)

#

====>no checkpoint found.


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0, Training Loss:  0.3058282136917114


 20%|██        | 1/5 [33:41<2:14:47, 2021.87s/it]

Epoch: 0, Validation Loss:  0.35
Epoch: 1, Training Loss:  0.2795849144458771


...

 60%|██████    | 3/5 [1:41:13<1:07:29, 2024.93s/it]

Epoch: 2, Validation Loss:  0.30
Epoch: 3, Training Loss:  0.20607627928256989


 80%|████████  | 4/5 [2:14:58<33:45, 2025.09s/it]  

Epoch: 3, Validation Loss:  0.28
Epoch: 4, Training Loss:  0.2069554328918457
Epoch: 4, Validation Loss:  0.31


100%|██████████| 5/5 [2:48:47<00:00, 2025.51s/it]


In [ ]:
DIR = '/content/drive/MyDrive/Colab Notebooks/MSc-Individual-Project/'

checkpoint = torch.load(f"%s/models/model_allnotesshuffle10_epoch5.pth" % DIR)
model.load_state_dict(checkpoint['model_state_dict'])



<All keys matched successfully>

In [ ]:
# Evaluate the model

def evaluation():
    model.eval()

    fin_targets=[]
    fin_outputs=[]
    losses=[]
    with torch.no_grad():
        for _, data in enumerate(dev_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            fin_targets.extend(targets.cpu().detach().numpy())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy())
    print(f'Loss:  {np.mean(losses):.2f}')
    return fin_outputs, fin_targets, losses

In [ ]:
dev_out, dev_tar, losses = evaluation()

Loss:  0.31


### Normal evaluation

In [ ]:
outputs = np.array(dev_out) >= 0.5
targets = dev_tar
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.792575789856253
F1 Score (Macro) = 0.7849589723825622


In [ ]:
print(classification_report(targets, outputs, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.7702    0.7536    0.7618      3015
       272.4     0.7967    0.7279    0.7608      2962
       401.9     0.8049    0.8185    0.8116      5807
      414.01     0.7995    0.8278    0.8134      3363
      427.31     0.7417    0.8929    0.8103      4640
       428.0     0.8156    0.8376    0.8265      4595
      518.81     0.8610    0.8392    0.8500      5434
      530.81     0.8767    0.7195    0.7903      2046
       584.9     0.7825    0.7795    0.7810      4436
       599.0     0.5415    0.7940    0.6439      2738

   micro avg     0.7758    0.8100    0.7926     39036
   macro avg     0.7790    0.7990    0.7850     39036
weighted avg     0.7854    0.8100    0.7946     39036
 samples avg     0.7620    0.7987    0.7589     39036



In [ ]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, outputs, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, outputs, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.8591435310273728
RUC AUC Score (Macro) = 0.8522578878445151


In [ ]:
dev_df['prediction'] = dev_out
dev_df['tar'] = dev_tar

In [ ]:
dev_df

,id,text,labels,prediction,tar
0,133208,npn neuro very lightly sedated on propofol and...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.050001696, 0.00018370486, 0.14803314, 0.001...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,105501,csru update neuro pt sedate on low dose propof...,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0]","[0.017103247, 0.00013826253, 0.0041555595, 0.4...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, ..."
2,155865,am shoulder ap neutral axillary soft tissue ri...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
3,165698,chief complaint hour events shovelmask transit...,"[0, 0, 1, 0, 0, 1, 1, 0, 1, 0]","[0.030178614, 0.011717622, 0.8371337, 0.981829...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, ..."
4,168578,subjective intubated objective height admit we...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0.00070152094, 0.0031774866, 0.002869218, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
...,...,...,...,...,...
13855,167558,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
13856,109739,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
13857,160172,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
13858,167925,admission date discharge date date of birth se...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.46759075, 0.098821074, 0.92293966, 0.135398...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### Avg outputs

In [ ]:
out_mean_dict = dev_df.groupby('id').prediction.apply(np.mean).to_dict()
dev_df['out_mean'] = dev_df['id'].map(out_mean_dict)
dev_df

,id,text,labels,prediction,tar,out_mean
0,133208,npn neuro very lightly sedated on propofol and...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.050001696, 0.00018370486, 0.14803314, 0.001...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.16991663, 0.1315904, 0.5986871, 0.07690332,..."
1,105501,csru update neuro pt sedate on low dose propof...,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0]","[0.017103247, 0.00013826253, 0.0041555595, 0.4...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.23221809, 0.010903329, 0.22674157, 0.445255..."
2,155865,am shoulder ap neutral axillary soft tissue ri...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.010249981, 0.0043252753, 0.003545368, 0.002..."
3,165698,chief complaint hour events shovelmask transit...,"[0, 0, 1, 0, 0, 1, 1, 0, 1, 0]","[0.030178614, 0.011717622, 0.8371337, 0.981829...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...","[0.01137908, 0.0041343793, 0.95452225, 0.25202..."
4,168578,subjective intubated objective height admit we...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0.00070152094, 0.0031774866, 0.002869218, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.05645007, 0.070297696, 0.21849471, 0.036902..."
...,...,...,...,...,...,...
13855,167558,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.045286708, 0.0047914227, 0.769825, 0.000391..."
13856,109739,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.6065295, 0.042903114, 0.04056662, 0.0191427..."
13857,160172,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.012064993, 0.008387041, 0.7498697, 0.000260..."
13858,167925,admission date discharge date date of birth se...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.46759075, 0.098821074, 0.92293966, 0.135398...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.46759075, 0.098821074, 0.92293966, 0.135398..."


In [ ]:
df_mean = dev_df.drop_duplicates('id')

In [ ]:
loss_mean = [nn.BCELoss()(torch.tensor(df_mean['out_mean'][i]), torch.tensor(df_mean['tar'][i])) for i in df_mean.index]
np.mean(loss_mean)

0.39683807

In [ ]:
out_mean = np.vstack([df_mean['out_mean'][i]>=0.5 for i in df_mean.index])
targets = np.vstack([df_mean['tar'][i] for i in df_mean.index])
#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_mean)
f1_score_micro = metrics.f1_score(targets, out_mean, average='micro')
f1_score_macro = metrics.f1_score(targets, out_mean, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6972166450139757
F1 Score (Macro) = 0.6820072898925524


In [ ]:
print(classification_report(targets, out_mean, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.6763    0.6310    0.6529      1000
       272.4     0.7407    0.6286    0.6801      1104
       401.9     0.7603    0.7723    0.7662      2279
      414.01     0.7692    0.7902    0.7795      1282
      427.31     0.6358    0.8408    0.7241      1470
       428.0     0.7334    0.7705    0.7515      1503
      518.81     0.7080    0.6739    0.6906      1058
      530.81     0.8133    0.5328    0.6438       687
       584.9     0.6506    0.6076    0.6284      1180
       599.0     0.4016    0.6734    0.5031       842

   micro avg     0.6814    0.7138    0.6972     12405
   macro avg     0.6889    0.6921    0.6820     12405
weighted avg     0.6984    0.7138    0.6998     12405
 samples avg     0.6729    0.7092    0.6604     12405



In [ ]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_mean, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_mean, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7978144904557066
RUC AUC Score (Macro) = 0.7844724729462073


### Most freq 3 labels

In [ ]:
out_sum_dict = dev_df.groupby('id').prediction.apply(np.sum).to_dict()
dev_df['out_sum'] = dev_df['id'].map(out_sum_dict)
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum
0,133208,npn neuro very lightly sedated on propofol and...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.050001696, 0.00018370486, 0.14803314, 0.001...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.16991663, 0.1315904, 0.5986871, 0.07690332,...","[1.0194998, 0.7895424, 3.5921228, 0.4614199, 0..."
1,105501,csru update neuro pt sedate on low dose propof...,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0]","[0.017103247, 0.00013826253, 0.0041555595, 0.4...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.23221809, 0.010903329, 0.22674157, 0.445255...","[0.92887235, 0.043613315, 0.90696627, 1.781022..."
2,155865,am shoulder ap neutral axillary soft tissue ri...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0.010249981, 0.0043252753, 0.003545368, 0.002..."
3,165698,chief complaint hour events shovelmask transit...,"[0, 0, 1, 0, 0, 1, 1, 0, 1, 0]","[0.030178614, 0.011717622, 0.8371337, 0.981829...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...","[0.01137908, 0.0041343793, 0.95452225, 0.25202...","[0.04551632, 0.016537517, 3.818089, 1.0081086,..."
4,168578,subjective intubated objective height admit we...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0.00070152094, 0.0031774866, 0.002869218, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.05645007, 0.070297696, 0.21849471, 0.036902...","[0.3387004, 0.42178616, 1.3109683, 0.22141388,..."
...,...,...,...,...,...,...,...
13855,167558,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0.045286708, 0.0047914227, 0.769825, 0.000391..."
13856,109739,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[0.6065295, 0.042903114, 0.04056662, 0.0191427..."
13857,160172,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[0.012064993, 0.008387041, 0.7498697, 0.000260..."
13858,167925,admission date discharge date date of birth se...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.46759075, 0.098821074, 0.92293966, 0.135398...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.46759075, 0.098821074, 0.92293966, 0.135398...","[0.46759075, 0.098821074, 0.92293966, 0.135398..."


In [ ]:
def freq_3(df, column): # column: out_sum
    df['freq_3'] = df[column]
    for idx in df.index:
      sorted = np.sort(df[column][idx])
      thres = sorted[-3] # position 3
      df['freq_3'][idx] = df[column][idx]>= thres


In [ ]:
freq_3(dev_df, 'out_sum')

In [ ]:
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum,freq_3
0,133208,npn neuro very lightly sedated on propofol and...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.050001696, 0.00018370486, 0.14803314, 0.001...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.16991663, 0.1315904, 0.5986871, 0.07690332,...","[1.0194998, 0.7895424, 3.5921228, 0.4614199, 0...","[False, False, True, False, False, False, True..."
1,105501,csru update neuro pt sedate on low dose propof...,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0]","[0.017103247, 0.00013826253, 0.0041555595, 0.4...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.23221809, 0.010903329, 0.22674157, 0.445255...","[0.92887235, 0.043613315, 0.90696627, 1.781022...","[False, False, False, False, True, True, True,..."
2,155865,am shoulder ap neutral axillary soft tissue ri...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[False, False, False, False, True, False, True..."
3,165698,chief complaint hour events shovelmask transit...,"[0, 0, 1, 0, 0, 1, 1, 0, 1, 0]","[0.030178614, 0.011717622, 0.8371337, 0.981829...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...","[0.01137908, 0.0041343793, 0.95452225, 0.25202...","[0.04551632, 0.016537517, 3.818089, 1.0081086,...","[False, False, False, False, False, True, True..."
4,168578,subjective intubated objective height admit we...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0.00070152094, 0.0031774866, 0.002869218, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.05645007, 0.070297696, 0.21849471, 0.036902...","[0.3387004, 0.42178616, 1.3109683, 0.22141388,...","[False, False, True, False, False, False, True..."
...,...,...,...,...,...,...,...,...
13855,167558,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[False, False, True, False, False, False, True..."
13856,109739,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[True, False, False, False, True, False, False..."
13857,160172,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[False, False, True, False, False, False, True..."
13858,167925,admission date discharge date date of birth se...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.46759075, 0.098821074, 0.92293966, 0.135398...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.46759075, 0.098821074, 0.92293966, 0.135398...","[0.46759075, 0.098821074, 0.92293966, 0.135398...","[True, False, True, False, False, False, False..."


In [ ]:
df_freq_3 = dev_df.drop_duplicates('id')

In [ ]:
out_freq_3 = np.vstack([df_freq_3['freq_3'][i] for i in df_freq_3.index])
targets = np.vstack([df_freq_3['tar'][i] for i in df_freq_3.index])
#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_freq_3)
f1_score_micro = metrics.f1_score(targets, out_freq_3, average='micro')
f1_score_macro = metrics.f1_score(targets, out_freq_3, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6219919660622442
F1 Score (Macro) = 0.6024596196927261


In [ ]:
print(classification_report(targets, out_freq_3, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.5441    0.5430    0.5435      1000
       272.4     0.5761    0.5145    0.5435      1104
       401.9     0.6926    0.7534    0.7217      2279
      414.01     0.7188    0.7176    0.7182      1282
      427.31     0.5522    0.8027    0.6543      1470
       428.0     0.6558    0.7212    0.6869      1503
      518.81     0.5812    0.6531    0.6150      1058
      530.81     0.6181    0.4571    0.5255       687
       584.9     0.5704    0.5805    0.5754      1180
       599.0     0.3232    0.6912    0.4404       842

   micro avg     0.5821    0.6678    0.6220     12405
   macro avg     0.5832    0.6434    0.6025     12405
weighted avg     0.6015    0.6678    0.6260     12405
 samples avg     0.5821    0.7183    0.6002     12405



In [ ]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_freq_3, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_freq_3, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7490110798972434
RUC AUC Score (Macro) = 0.7338456819537021


### Predicted percentage

In [ ]:
note_count_dict = dev_df.groupby('id').size().to_dict()
dev_df['note_count'] = dev_df['id'].map(note_count_dict)

In [ ]:
dev_df['out_bool'] = [(dev_df['prediction'][i]>=0.5).astype(int) for i in dev_df.index]

In [ ]:

out_freq_dict = dev_df.groupby('id').out_bool.apply(np.sum).to_dict()
dev_df['num_pred'] = dev_df['id'].map(out_freq_dict)
dev_df['num_pred'] = [(dev_df['num_pred'][i]>=0.6*dev_df['note_count'][i]).astype(int) for i in dev_df.index]

In [ ]:
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum,freq_3,note_count,out_bool,num_pred
0,133208,npn neuro very lightly sedated on propofol and...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.050001696, 0.00018370486, 0.14803314, 0.001...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.16991663, 0.1315904, 0.5986871, 0.07690332,...","[1.0194998, 0.7895424, 3.5921228, 0.4614199, 0...","[False, False, True, False, False, False, True...",6,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0]","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]"
1,105501,csru update neuro pt sedate on low dose propof...,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0]","[0.017103247, 0.00013826253, 0.0041555595, 0.4...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.23221809, 0.010903329, 0.22674157, 0.445255...","[0.92887235, 0.043613315, 0.90696627, 1.781022...","[False, False, False, False, True, True, True,...",4,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]"
2,155865,am shoulder ap neutral axillary soft tissue ri...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[False, False, False, False, True, False, True...",1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
3,165698,chief complaint hour events shovelmask transit...,"[0, 0, 1, 0, 0, 1, 1, 0, 1, 0]","[0.030178614, 0.011717622, 0.8371337, 0.981829...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...","[0.01137908, 0.0041343793, 0.95452225, 0.25202...","[0.04551632, 0.016537517, 3.818089, 1.0081086,...","[False, False, False, False, False, True, True...",4,"[0, 0, 1, 1, 0, 1, 1, 0, 1, 0]","[0, 0, 1, 0, 0, 1, 1, 0, 1, 0]"
4,168578,subjective intubated objective height admit we...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0.00070152094, 0.0031774866, 0.002869218, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.05645007, 0.070297696, 0.21849471, 0.036902...","[0.3387004, 0.42178616, 1.3109683, 0.22141388,...","[False, False, True, False, False, False, True...",6,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
...,...,...,...,...,...,...,...,...,...,...,...
13855,167558,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[False, False, True, False, False, False, True...",1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
13856,109739,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[True, False, False, False, True, False, False...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
13857,160172,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[False, False, True, False, False, False, True...",1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]"
13858,167925,admission date discharge date date of birth se...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.46759075, 0.098821074, 0.92293966, 0.135398...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.46759075, 0.098821074, 0.92293966, 0.135398...","[0.46759075, 0.098821074, 0.92293966, 0.135398...","[True, False, True, False, False, False, False...",1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
df_freq = dev_df.drop_duplicates('id')

In [ ]:
out_freq = np.vstack([df_freq['num_pred'][i] for i in df_freq.index])
targets = np.vstack([df_freq['tar'][i] for i in df_freq.index])

#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_freq)
f1_score_micro = metrics.f1_score(targets, out_freq, average='micro')
f1_score_macro = metrics.f1_score(targets, out_freq, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6795170861469204
F1 Score (Macro) = 0.660865736161891


In [ ]:
print(classification_report(targets, out_freq, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.6753    0.5760    0.6217      1000
       272.4     0.7270    0.5788    0.6445      1104
       401.9     0.7655    0.7420    0.7536      2279
      414.01     0.7760    0.7621    0.7690      1282
      427.31     0.6437    0.8014    0.7139      1470
       428.0     0.7428    0.7206    0.7315      1503
      518.81     0.7196    0.6163    0.6640      1058
      530.81     0.8131    0.4876    0.6096       687
       584.9     0.6579    0.5525    0.6006      1180
       599.0     0.4209    0.6164    0.5002       842

   micro avg     0.6901    0.6692    0.6795     12405
   macro avg     0.6942    0.6454    0.6609     12405
weighted avg     0.7038    0.6692    0.6801     12405
 samples avg     0.6596    0.6639    0.6301     12405



In [ ]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_freq, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_freq, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7814191969141011
RUC AUC Score (Macro) = 0.76706330838122


### Exponential moving average

In [ ]:
def ewma(sub_df, window=3):
#    print(sub_df)
    alpha = 2 / (window + 1)
#    print(sub_df['ewma'])
    sub_df['ewma'] = sub_df['out_bool']
    for r in sub_df.index:
        if r == sub_df.index[0]:
            sub_df['ewma'][r] = sub_df['prediction'][r]
        else:
            sub_df['ewma'][r] = alpha*sub_df['prediction'][r] + (1-alpha)*sub_df['prediction'][sub_df.index[sub_df.index.get_loc(r)-1]]
 #   print(type(sub_df['ewma']))
    return sub_df['ewma']

In [ ]:

out_ewma_dict = dev_df.groupby('id', group_keys=False).apply(ewma).to_dict()
dev_df['out_ewma'] = pd.Series(dev_df.index, index=dev_df.index).map(out_ewma_dict)



In [ ]:
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum,freq_3,note_count,out_bool,num_pred,out_ewma
0,133208,npn neuro very lightly sedated on propofol and...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.050001696, 0.00018370486, 0.14803314, 0.001...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.16991663, 0.1315904, 0.5986871, 0.07690332,...","[1.0194998, 0.7895424, 3.5921228, 0.4614199, 0...","[False, False, True, False, False, False, True...",6,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0]","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0.050001696, 0.00018370486, 0.14803314, 0.001..."
1,105501,csru update neuro pt sedate on low dose propof...,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0]","[0.017103247, 0.00013826253, 0.0041555595, 0.4...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.23221809, 0.010903329, 0.22674157, 0.445255...","[0.92887235, 0.043613315, 0.90696627, 1.781022...","[False, False, False, False, True, True, True,...",4,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]","[0.017103247, 0.00013826253, 0.0041555595, 0.4..."
2,155865,am shoulder ap neutral axillary soft tissue ri...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[False, False, False, False, True, False, True...",1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.010249981, 0.0043252753, 0.003545368, 0.002..."
3,165698,chief complaint hour events shovelmask transit...,"[0, 0, 1, 0, 0, 1, 1, 0, 1, 0]","[0.030178614, 0.011717622, 0.8371337, 0.981829...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...","[0.01137908, 0.0041343793, 0.95452225, 0.25202...","[0.04551632, 0.016537517, 3.818089, 1.0081086,...","[False, False, False, False, False, True, True...",4,"[0, 0, 1, 1, 0, 1, 1, 0, 1, 0]","[0, 0, 1, 0, 0, 1, 1, 0, 1, 0]","[0.030178614, 0.011717622, 0.8371337, 0.981829..."
4,168578,subjective intubated objective height admit we...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0.00070152094, 0.0031774866, 0.002869218, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.05645007, 0.070297696, 0.21849471, 0.036902...","[0.3387004, 0.42178616, 1.3109683, 0.22141388,...","[False, False, True, False, False, False, True...",6,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0.00070152094, 0.0031774866, 0.002869218, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...
13855,167558,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[False, False, True, False, False, False, True...",1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.045286708, 0.0047914227, 0.769825, 0.000391..."
13856,109739,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[True, False, False, False, True, False, False...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.6065295, 0.042903114, 0.04056662, 0.0191427..."
13857,160172,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[False, False, True, False, False, False, True...",1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]","[0.012064993, 0.008387041, 0.7498697, 0.000260..."
13858,167925,admission date discharge date date of birth se...,"[1,

In [ ]:
df_ewma = dev_df.drop_duplicates('id')

In [ ]:
out_ewma = np.vstack([df_ewma['out_ewma'][i]>0.5 for i in df_ewma.index])
targets = np.vstack([df_ewma['tar'][i] for i in df_ewma.index])

#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_ewma)
f1_score_micro = metrics.f1_score(targets, out_ewma, average='micro')
f1_score_macro = metrics.f1_score(targets, out_ewma, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6449173682569379
F1 Score (Macro) = 0.6248752510524431


In [ ]:
print(classification_report(targets, out_ewma, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.5996    0.5630    0.5807      1000
       272.4     0.6553    0.5734    0.6116      1104
       401.9     0.7149    0.7473    0.7307      2279
      414.01     0.7093    0.7480    0.7282      1282
      427.31     0.5881    0.8102    0.6815      1470
       428.0     0.6912    0.7312    0.7106      1503
      518.81     0.6429    0.6144    0.6283      1058
      530.81     0.7458    0.4527    0.5634       687
       584.9     0.5790    0.5525    0.5655      1180
       599.0     0.3548    0.6081    0.4481       842

   micro avg     0.6243    0.6669    0.6449     12405
   macro avg     0.6281    0.6401    0.6249     12405
weighted avg     0.6400    0.6669    0.6466     12405
 samples avg     0.6152    0.6644    0.6052     12405



In [ ]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_ewma, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_ewma, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7624110098457999
RUC AUC Score (Macro) = 0.7458994575084799


### Most frequent prediction

In [ ]:

most_freq_dict = dev_df.groupby('id')['out_bool'].apply(lambda x: x.value_counts().index[0]).to_dict()
dev_df['most_freq'] = dev_df['id'].map(most_freq_dict)

In [ ]:
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum,freq_3,note_count,out_bool,num_pred,out_ewma,most_freq
0,133208,npn neuro very lightly sedated on propofol and...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.050001696, 0.00018370486, 0.14803314, 0.001...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.16991663, 0.1315904, 0.5986871, 0.07690332,...","[1.0194998, 0.7895424, 3.5921228, 0.4614199, 0...","[False, False, True, False, False, False, True...",6,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0]","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0.050001696, 0.00018370486, 0.14803314, 0.001...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
1,105501,csru update neuro pt sedate on low dose propof...,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0]","[0.017103247, 0.00013826253, 0.0041555595, 0.4...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.23221809, 0.010903329, 0.22674157, 0.445255...","[0.92887235, 0.043613315, 0.90696627, 1.781022...","[False, False, False, False, True, True, True,...",4,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]","[0.017103247, 0.00013826253, 0.0041555595, 0.4...","[1, 0, 0, 0, 1, 1, 1, 0, 0, 1]"
2,155865,am shoulder ap neutral axillary soft tissue ri...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[False, False, False, False, True, False, True...",1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.010249981, 0.0043252753, 0.003545368, 0.002...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
3,165698,chief complaint hour events shovelmask transit...,"[0, 0, 1, 0, 0, 1, 1, 0, 1, 0]","[0.030178614, 0.011717622, 0.8371337, 0.981829...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...","[0.01137908, 0.0041343793, 0.95452225, 0.25202...","[0.04551632, 0.016537517, 3.818089, 1.0081086,...","[False, False, False, False, False, True, True...",4,"[0, 0, 1, 1, 0, 1, 1, 0, 1, 0]","[0, 0, 1, 0, 0, 1, 1, 0, 1, 0]","[0.030178614, 0.011717622, 0.8371337, 0.981829...","[0, 0, 1, 0, 0, 1, 1, 0, 1, 0]"
4,168578,subjective intubated objective height admit we...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0.00070152094, 0.0031774866, 0.002869218, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.05645007, 0.070297696, 0.21849471, 0.036902...","[0.3387004, 0.42178616, 1.3109683, 0.22141388,...","[False, False, True, False, False, False, True...",6,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0.00070152094, 0.0031774866, 0.002869218, 0.0...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13855,167558,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[False, False, True, False, False, False, True...",1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.045286708, 0.0047914227, 0.769825, 0.000391...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
13856,109739,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[True, False, False, False, True, False, False...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.6065295, 0.042903114, 0.04056662, 0.0191427...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
13857,160172,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.012064993, 0.008387041, 0.7498697, 0.000260...","[0.012064993, 0.008387041, 0.7498697, 0.00026

In [ ]:
df_most_freq = dev_df.drop_duplicates('id')

In [ ]:
out_most_freq = np.vstack([df_most_freq['most_freq'][i] for i in df_most_freq.index])
targets = np.vstack([df_most_freq['tar'][i] for i in df_most_freq.index])
#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_most_freq)
f1_score_micro = metrics.f1_score(targets, out_most_freq, average='micro')
f1_score_macro = metrics.f1_score(targets, out_most_freq, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6587119592677524
F1 Score (Macro) = 0.6400109551434996


In [ ]:
print(classification_report(targets, out_most_freq, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.6114    0.5900    0.6005      1000
       272.4     0.6728    0.5942    0.6311      1104
       401.9     0.7240    0.7573    0.7403      2279
      414.01     0.7189    0.7660    0.7417      1282
      427.31     0.5994    0.8204    0.6927      1470
       428.0     0.6994    0.7478    0.7228      1503
      518.81     0.6571    0.6267    0.6415      1058
      530.81     0.7574    0.4818    0.5890       687
       584.9     0.5954    0.5763    0.5857      1180
       599.0     0.3616    0.6128    0.4548       842

   micro avg     0.6360    0.6831    0.6587     12405
   macro avg     0.6398    0.6573    0.6400     12405
weighted avg     0.6514    0.6831    0.6608     12405
 samples avg     0.6279    0.6772    0.6186     12405



In [ ]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_most_freq, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_most_freq, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7723393264764831
RUC AUC Score (Macro) = 0.7564693229220864


### Testing

In [ ]:

# Evaluate the model

def testing():
    model.eval()

    fin_targets=[]
    fin_outputs=[]
    losses=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            fin_targets.extend(targets.cpu().detach().numpy())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy())
    print(f'Loss:  {np.mean(losses):.2f}')
    return fin_outputs, fin_targets, losses

In [ ]:
test_out, targets, losses = testing()
outputs = np.array(test_out) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Loss:  0.30
Accuracy Score = 0.5025685425685426
F1 Score (Micro) = 0.7994940419423168
F1 Score (Macro) = 0.7909820621467333


In [ ]:
print(classification_report(targets, outputs, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.7756    0.7561    0.7657      3735
       272.4     0.8042    0.7270    0.7637      3660
       401.9     0.8060    0.8328    0.8192      7094
      414.01     0.8153    0.8441    0.8295      4132
      427.31     0.7523    0.8947    0.8173      5791
       428.0     0.8262    0.8323    0.8293      5879
      518.81     0.8722    0.8450    0.8584      6944
      530.81     0.8686    0.7200    0.7874      2654
       584.9     0.7924    0.7861    0.7892      5559
       599.0     0.5458    0.8038    0.6501      3379

   micro avg     0.7841    0.8155    0.7995     48827
   macro avg     0.7859    0.8042    0.7910     48827
weighted avg     0.7937    0.8155    0.8015     48827
 samples avg     0.7692    0.8033    0.7652     48827



In [ ]:

ruc_auc_score_micro = metrics.roc_auc_score(targets, outputs, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, outputs, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.8636987797446211
RUC AUC Score (Macro) = 0.8567955759077922


In [ ]:
test_df['prediction'] = test_out
test_df['tar'] = targets

In [ ]:
out_mean_dict = test_df.groupby('id').prediction.apply(np.mean).to_dict()
test_df['out_mean'] = test_df['id'].map(out_mean_dict)
test_df

,id,text,labels,prediction,tar,out_mean
0,111336,year old male admitted s p fall from a ladder ...,"[0, 1, 1, 0, 0, 0, 1, 0, 0, 0]","[0.004077224, 0.9997707, 0.9987457, 0.00636857...","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.030306978, 0.9658669, 0.97903436, 0.0154112..."
1,192003,this is a year old female with a history of ci...,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0]","[0.0004029054, 0.0001446614, 0.0008918275, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[0.0296192, 0.009818944, 0.02741521, 0.0278871..."
2,153856,am last name un intestinal tube placement w fl...,"[0, 1, 1, 0, 1, 1, 0, 0, 0, 0]","[0.23453711, 0.76273423, 0.10356724, 0.3752519...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[0.23453711, 0.76273423, 0.10356724, 0.3752519..."
3,181811,coronary artery bypass graft cabg assessment p...,"[0, 1, 1, 1, 1, 1, 0, 0, 0, 0]","[0.45701206, 0.2910865, 0.81099695, 0.99946004...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[0.15652527, 0.37404165, 0.8810051, 0.95523816..."
4,117776,nursing progress note pt continues to have res...,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0]","[0.0897669, 0.023824912, 0.47742677, 0.2931985...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.3115778, 0.18635549, 0.6143979, 0.5704639, ..."
...,...,...,...,...,...,...
17320,135280,admission date discharge date date of birth se...,"[0, 0, 0, 1, 0, 0, 1, 0, 1, 0]","[0.7894151, 0.21467474, 0.47898072, 0.9953458,...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[0.4052021, 0.09983738, 0.14770639, 0.97482926..."
17321,173920,admission date discharge date date of birth se...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.97947127, 0.056570973, 0.12766916, 0.015893...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9716811, 0.014070407, 0.026008893, 0.020153..."
17322,181850,admission date discharge date date of birth se...,"[0, 1, 1, 0, 1, 1, 0, 0, 0, 0]","[0.05532062, 0.035659075, 0.73378706, 0.928531...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[0.05532062, 0.035659075, 0.73378706, 0.928531..."
17323,144980,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0.043764815, 0.015764823, 0.93055713, 0.02186...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.18672803, 0.17150113, 0.72948766, 0.1607179..."


In [ ]:
loss_mean = [nn.BCELoss()(torch.tensor(test_df['out_mean'][i]), torch.tensor(test_df['tar'][i])) for i in test_df.index]
np.mean(loss_mean)

0.21728916

In [ ]:
df_mean = test_df.drop_duplicates('id')

In [ ]:
out_mean = np.vstack([df_mean['out_mean'][i]>=0.5 for i in df_mean.index])
targets = np.vstack([df_mean['tar'][i] for i in df_mean.index])
#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_mean)
f1_score_micro = metrics.f1_score(targets, out_mean, average='micro')
f1_score_macro = metrics.f1_score(targets, out_mean, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6948028281944749
F1 Score (Macro) = 0.6795467825168041


In [ ]:
print(classification_report(targets, out_mean, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      250.00     0.6960    0.5930    0.6404      1108
       272.4     0.7289    0.6102    0.6643      1216
       401.9     0.7551    0.7805    0.7676      2465
      414.01     0.7814    0.7871    0.7842      1376
      427.31     0.6431    0.8355    0.7268      1611
       428.0     0.7196    0.7508    0.7349      1617
      518.81     0.7093    0.6702    0.6892      1125
      530.81     0.8236    0.5359    0.6494       793
       584.9     0.6630    0.6091    0.6349      1269
       599.0     0.4026    0.6732    0.5039       912

   micro avg     0.6835    0.7065    0.6948     13492
   macro avg     0.6923    0.6846    0.6795     13492
weighted avg     0.7007    0.7065    0.6969     13492
 samples avg     0.6708    0.7056    0.6581     13492



In [ ]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_mean, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_mean, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7954079070829237
RUC AUC Score (Macro) = 0.7818994854857293
